<a href="https://colab.research.google.com/github/MUZAKI1453/Convolutional-Neural-Networks/blob/main/Klasifikasi_gambar_abjad_a_dan_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mengambil Dataset dari drive google / mounting

In [ ]:
from google.colab import drive
# mounting ke google drive
drive.mount('/content/drive')

# menentukan path ke folder dataset
dataset_path = '/content/drive/MyDrive/DataScience/Dataset_abjad'

# memeriksa apakah folder dan dataset ada
!ls $dataset_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
abjad_a  abjad_b


install dan import library yang dibutuhkan

In [ ]:
!pip install tensorflow opencv-python scikit-learn

import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Preprocessing Data

In [ ]:
# menentukan ukuran gambar yang seragam
img_size = 64

# membaca dataset dan menyiapkan data
def load_data(data_dir):
  data = []
  labels = []
  for label, folder in enumerate(os.listdir(data_dir)):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                # baca gambar dalam grayscale
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                # ubah ukuran gambar
                img = cv2.resize(img, (img_size, img_size))
                data.append(img)
                labels.append(label)

  data = np.array(data)
  labels = np.array(labels)

  # normalisasi gambar agar nilai piksel antara o dan 1
  data = data / 255.0

  # ubah label menjadi one-hot encoding
  labels = to_categorical(labels)

  return data, labels

# Path dataset
dataset_path = '/content/drive/MyDrive/DataScience/Dataset_abjad'
data, labels = load_data(dataset_path)

# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Ubah bentuk data agar sesuai dengan input model CNN
X_train = X_train.reshape(-1, img_size, img_size, 1)
X_test = X_test.reshape(-1, img_size, img_size, 1)


Bangun model cnn

In [ ]:
# Bangun model CNN
model = Sequential()

# Layer konvolusi pertama
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)))
model.add(MaxPooling2D((2, 2)))

# Layer konvolusi kedua
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Layer konvolusi ketiga
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Ratakan hasil untuk input ke layer dense
model.add(Flatten())

# Layer dense (fully connected)
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 karena ada 2 kelas (a dan b)

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Ringkasan model
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_27 (Conv2D)                   │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_28 (MaxPooling2D)      │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_29 (MaxPooling2D)      │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 682,882 (2.60 MB)

 Trainable params: 682,882 (2.60 MB)

 Non-trainable params: 0 (0.00 B)

Latih Model

In [ ]:
# Latih model
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.3018 - loss: 0.9301 - val_accuracy: 0.4286 - val_loss: 0.6924
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6595 - loss: 0.6800 - val_accuracy: 0.5714 - val_loss: 0.6774
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5125 - loss: 0.6816 - val_accuracy: 0.4286 - val_loss: 0.7042
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7649 - loss: 0.6316 - val_accuracy: 0.4286 - val_loss: 0.6805
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7488 - loss: 0.5854 - val_accuracy: 0.5714 - val_loss: 0.6554
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9202 - loss: 0.4613 - val_accuracy: 0.5714 - val_loss: 0.6466
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7851 - loss: 0.4298 - val_accuracy: 0.8571 - val_loss: 0.5019
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8905 - loss: 0.2571 - val_accuracy: 1.0000 - val_loss: 0.2912


Evaluasi dan Prediksi

In [ ]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Prediksi
predictions = model.predict(X_test)

# Tampilkan prediksi untuk contoh pertama
print("Prediksi untuk contoh pertama:", np.argmax(predictions[0]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 1.0000 - loss: 0.1670
Accuracy: 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Prediksi untuk contoh pertama: 1


memprediksi gambar

In [ ]:
img_path = '/content/drive/MyDrive/DataScience/test/test6.PNG'

# membaca gambar dalam grayscale
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Mengubah ukuran gambar ke ukuran input model (misalnya 64x64)
img = cv2.resize(img, (img_size, img_size))

# Normalisasi nilai piksel antara 0 dan 1
img = img / 255.0

# Menambah batch dimension
img = np.expand_dims(img, axis=0)

# Menambah channel dimension (grayscale)
img = np.expand_dims(img, axis=-1)

# Prediksi dengan model
predictions = model.predict(img)

# Menampilkan hasil prediksi
predicted_class = np.argmax(predictions)  # Mengambil kelas dengan probabilitas tertinggi

# Output hasil prediksi
if predicted_class == 0:
        print("Ini adalah huruf 'a'.")
elif predicted_class == 1:
        print("Ini adalah huruf 'b'.")
else:
        print("Prediksi lainnya:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Ini adalah huruf 'b'.
